In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split

df= pd.read_csv("data/chosen_features.csv")
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape

(11228, 61)

In [62]:
from sklearn.metrics import classification_report

def generate_metric_report(y_true, y_pred):
    report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True))
    report.iloc[:-1, :] = report.iloc[:-1, :] *100
    for i, col in enumerate(report.columns):
        report[col] = report[col].astype(str)
        report.iloc[:-1, i] = report.iloc[:-1, i].apply( lambda v: str(f"{ float(v):.2f} %" ) )
    return report

In [15]:
from sklearn.svm import SVC
clf_svc = SVC(kernel = "linear")
clf_svc.fit(x_train, y_train)
svc_pred = clf_svc.predict(x_test)


,anomaly,normal,accuracy,macro avg,weighted avg
precision,95.798319,99.628115,99.465812,97.713217,99.458993
recall,91.935484,99.813711,99.465812,95.874597,99.465812
f1-score,93.827160,99.720826,99.465812,96.773993,99.460565
support,12400.000000,268400.000000,99.465812,280800.000000,280800.000000


In [63]:
generate_metric_report(y_test, svc_pred)

,anomaly,normal,accuracy,macro avg,weighted avg
precision,95.80 %,99.63 %,99.47 %,97.71 %,99.46 %
recall,91.94 %,99.81 %,99.47 %,95.87 %,99.47 %
f1-score,93.83 %,99.72 %,99.47 %,96.77 %,99.46 %
support,124.0,2684.0,0.9946581196581197,2808.0,2808.0
